In [ ]:
# Arquivo: modelo_clusterizacao.ipynb

import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import joblib

# 1. Carregar dados
df = pd.read_csv('dados_recomece.csv')

# 2. Selecionar colunas para agrupar (Foco em vulnerabilidade social)
dados_cluster = df[['idade', 'tempo_desemprego', 'renda_familiar']]

# 3. Escalonar os dados (Importante para KMeans)
scaler = StandardScaler()
dados_scaled = scaler.fit_transform(dados_cluster)

# 4. Treinar KMeans (Vamos criar 3 grupos: Alta, Média e Baixa vulnerabilidade)
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(dados_scaled)

# 5. Adicionar o resultado ao dataframe para visualizar
df['grupo_social'] = kmeans.labels_
print(df.groupby('grupo_social')[['renda_familiar', 'tempo_desemprego']].mean())

# 6. Salvar modelo
joblib.dump(kmeans, 'modelo_cluster.joblib')
joblib.dump(scaler, 'scaler_cluster.joblib') # Precisamos salvar o escalador também
print("Modelos de cluster salvos.")